In [1]:
import os
import unsloth

os.environ["UNSLOTH_FORCE_FLOAT16"] = "1"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


To teach a model to use your MCP server, click **Runtime** > **Run all**. Make sure you've enabled a free Tesla T4 GPU and edit the [configuration](#configuration) cell below!

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord.png" height="50"></a>
<a href="https://art.openpipe.ai"><img src="https://github.com/openpipe/art/raw/main/assets/Documentation_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/openpipe/art).

</div>

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

**MCP•RL: Teach you agent how to use any MCP server**

This notebook shows how to train a Qwen 2.5 3B model to effectively use any MCP server. Simply provide an MCP server url and the notebook will:

1. Query the server's tools
2. Generate a set of input tasks that use those tools
3. Train the model on those tasks using automatic RULER evaluation
4. Test the trained model by giving it new tasks to complete

RULER judges response quality purely from the agent's final output - no labeled data required!

*Note: In this notebook we use local MCP servers, but the technique below applies to all MCP servers!*


In [2]:
# @title 💿 Installation
# Portions adapted from Unsloth Notebooks (https://github.com/unslothai/notebooks)
# Copyright (c) Unsloth contributors.
# License: GNU LGPL v3.0.
# Modifications by OpenPipe:
# - switched to uv
# - changed vllm/triton pinning logic
# - added protobuf pins
# - adjusted syntax for pushing to HF
# See /licenses/LGPL-3.0.txt and /licenses/GPL-3.0.txt for full text.

import os

if "COLAB_" not in "".join(os.environ.keys()):
    !uv pip install "openpipe-art[backend]==0.5.9" tenacity "mcp>=1.11.0" "gql<4" aiohttp --prerelease allow --no-cache-dir
else:
    try:
        import numpy

        get_numpy = f"numpy=={numpy.__version__}"
    except:
        get_numpy = "numpy"
    try:
        import subprocess

        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except:
        is_t4 = False
    get_vllm, get_triton = (
        ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm", "triton")
    )
    !uv pip install --upgrade \
        openpipe-art[backend]==0.4.11 tenacity pillow==11.3.0 protobuf==5.29.5 {get_vllm} {get_numpy} --prerelease allow --no-cache-dir
    !uv pip install -qqq {get_triton}

Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Audited 5 packages in 105ms


## MCP server 

In [3]:
!uv pip install fastmcp --quiet

In [2]:
import subprocess
subprocess.Popen(["python", "mcp_server.py"])

<Popen: returncode: None args: ['python', 'mcp_server.py']>



╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│                                                                              │
│                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │
│                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │
│                                                                              │
│                                                                              │
│                                FastMCP 2.14.5                                │
│                            https://gofastmcp.com                             │
│                                                                              │
│                    🖥  Server:      company-db                                │
│                    🚀 Deploy free: https://fastmcp.cloud                     │
│                          

<a name="configuration"></a>

### 🎯 Configuration - Edit These Settings

Add an OpenRouter API key and MCP server url below.

In [3]:
# Required - Used for generating training inputs and RULER evaluation
OPENROUTER_API_KEY = ""  # Put your OpenRouter key here

# 🔌 Point to any  MCP server
MCP_SERVER_URL = "http://0.0.0.0:8903/mcp"

# Optional - Enables metric logging
WANDB_API_KEY = ""

# Choose the base model to train
BASE_MODEL = "Qwen/Qwen2.5-3B-Instruct"  # Options: "Qwen/Qwen2.5-3B-Instruct", "Qwen/Qwen2.5-7B-Instruct", etc.

In [4]:
# @title Advanced Settings

# Model configuration
MODEL_NAME = "sql-agent-3b"  # Name for your trained model
PROJECT_NAME = "mcp-rl"  # Project name for tracking

# Training configuration
TRAINING_CONFIG = {
    "num_training_inputs": 16,  # Number of training inputs to generate
    "groups_per_step": 2,  # Inputs to process per training step
    "num_epochs": 1,  # Number of times through all data
    "rollouts_per_group": 4,  # Different responses per input (for RULER comparison)
    "learning_rate": 1e-5,  # Learning rate
    "max_training_steps": None,  # Maximum training steps (set to None for no limit)
}

MAX_TURNS = 10  # Maximum number of turns for the model to generate during one rollout

NUM_TEST_INPUTS = 8  # Number of test inputs to generate
RULER_MODEL = "openrouter/openai/gpt-4o-mini"  # Model for RULER evaluation
INPUT_GENERATION_MODEL = "openai/gpt-5-nano"

# Colab/T4 specific config to avoid OOM errors
MAX_TURNS = 3  # Decrease the number of turns to avoid OOM errors on a T4
MAX_SEQ_LENGTH = 16384  # Maximum sequence length
GPU_MEMORY_UTILIZATION = 0.8  # GPU memory usage (0.0-1.0)

In [5]:
# @title Debug utilities

import json
import time
import traceback
from typing import Any

DEBUG_LOG = True  # flip to False to silence logs
LOG_JSON_MAX = 2000  # cap large JSON prints


def _ts() -> str:
    return time.strftime("%H:%M:%S")


def log(msg: str, **kv):
    if not DEBUG_LOG:
        return
    parts = [f"[{_ts()}] {msg}"]
    if kv:
        kv_str = " ".join(f"{k}={repr(v)}" for k, v in kv.items())
        parts.append("| " + kv_str)
    print(" ".join(parts))


def log_json(title: str, payload: Any, max_len: int = LOG_JSON_MAX):
    if not DEBUG_LOG:
        return
    try:
        s = json.dumps(payload, indent=2, default=str)
    except Exception:
        s = str(payload)
    if len(s) > max_len:
        s = s[:max_len] + "\n... (truncated)"
    print(f"[{_ts()}] {title}:\n{s}")

In [6]:
# @title 🔌 MCP helpers

from contextlib import asynccontextmanager

import mcp.types as types
from mcp.client.session import ClientSession
from mcp.client.streamable_http import streamable_http_client


@asynccontextmanager
async def mcp_session():
    """
    Connects to the remote Smithery MCP server using the full URL that includes
    your API key & profile. No OAuth provider is used.
    """
    async with streamable_http_client(MCP_SERVER_URL) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            yield session


async def list_tools_and_resources():
    """Return (tools_result, resources_result) from the remote Smithery server."""
    async with mcp_session() as session:
        tools = await session.list_tools()
        try:
            resources = await session.list_resources()
        except Exception:
            # Some servers don't implement resources; keep interface stable
            class _Empty:
                resources = []

            resources = _Empty()
        return tools, resources


async def call_mcp_tool(tool_name: str, arguments: dict):
    """Invoke a tool on the remote Smithery server and return the CallToolResult."""
    async with mcp_session() as session:
        return await session.call_tool(tool_name, arguments)


tools, resources = await list_tools_and_resources()
print("Tools:", [t.name for t in tools.tools])
print(
    "Resources:",
    [getattr(r, "uri", None) for r in getattr(resources, "resources", []) or []],
)

INFO:     127.0.0.1:50804 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50810 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50818 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50826 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50840 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50854 - "DELETE /mcp HTTP/1.1" 200 OK
Tools: ['list_tables', 'describe_table', 'run_query']
Resources: []


In [7]:
!nvidia-smi

Mon Feb 16 18:45:24 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.211.01             Driver Version: 570.211.01     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:05.0 Off |                    0 |
| N/A   47C    P0             26W /   70W |     363MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
def get_content_text(result) -> str:
    # Extract text content from tool call result per MCP content schema
    if isinstance(result, str):
        return result
    if hasattr(result, "content") and result.content:
        out = ""
        for item in result.content:
            if isinstance(item, types.TextContent):
                out += item.text
            else:
                out += str(item)
        return out
    if hasattr(result, "structured") and result.structured is not None:
        try:
            return json.dumps(result.structured)
        except Exception:
            return str(result.structured)
    return str(result)

In [9]:
# @title Let's generate our train and validation scenarios!

import os
import random

from dotenv import load_dotenv

# Import the generate_scenarios function from art.mcp and logging utilities
from art.mcp import generate_scenarios
from art.mcp.generate_scenarios import preview_scenarios
from art.utils.logging import info, ok, step, warn, err

load_dotenv()

# required env/key check
# If OPENROUTER_API_KEY exists as a var, use it; otherwise pull from env
_openrouter_key = os.getenv("OPENROUTER_API_KEY")
try:
    _openrouter_key = _openrouter_key if _openrouter_key else OPENROUTER_API_KEY  # noqa: F821 (defined upstream in your notebook)
except NameError:
    pass

if _openrouter_key:
    os.environ["OPENROUTER_API_KEY"] = _openrouter_key
    ok("OPENROUTER_API_KEY found.")
else:
    err("OPENROUTER_API_KEY is required for data generation and RULER evaluation.")
    raise ValueError(
        "OPENROUTER_API_KEY is required for data generation and RULER evaluation."
    )

# Convert MCP tools and resources to the expected format
tools_result, resources_result = await list_tools_and_resources()

# Convert tools to the format expected by generate_scenarios
tools_list = []
for tool in tools_result.tools or []:
    tools_list.append({
        "name": tool.name,
        "description": tool.description,
        "parameters": tool.inputSchema,
    })

# Convert resources to the format expected by generate_scenarios
resources_list = []
for resource in getattr(resources_result, "resources", []) or []:
    resources_list.append({
        "uri": str(resource.uri),
        "name": resource.name,
        "description": resource.description,
        "mimeType": resource.mimeType,
    })

# First, get the actual schema from your database
schema_info = ""
for table in ["departments", "employees", "projects"]:
    result = await call_mcp_tool("describe_table", {"table_name": table})
    schema_info += f"\nTable '{table}': {get_content_text(result)}"

# Sample a few rows so the generator knows what kind of data exists
for table in ["departments", "employees", "projects"]:
    result = await call_mcp_tool("run_query", {"sql": f"SELECT * FROM {table} LIMIT 3"})
    schema_info += f"\nSample data from '{table}': {get_content_text(result)}"

# Now enrich the tool descriptions with this context
enriched_tools_list = []
for tool in tools_list:
    enriched = tool.copy()
    if tool["name"] == "run_query":
        enriched["description"] = (
            tool["description"] +
            f"\n\nAvailable database schema:{schema_info}"
        )
    enriched_tools_list.append(enriched)


# Calculate total scenarios needed
try:
    expected_total = TRAINING_CONFIG["num_training_inputs"] + NUM_TEST_INPUTS  # noqa: F821
except NameError:
    err("TRAINING_CONFIG/NUM_TEST_INPUTS not defined in this notebook.")
    raise

info(f"Target total scenarios: {expected_total}")

# Generate scenarios using the art.mcp function
max_attempts = 10
scenarios = None

for attempt in range(1, max_attempts + 1):
    step(f"Attempt {attempt}/{max_attempts} ...")
    t_attempt = time.perf_counter()
    try:
        scenario_collection = await generate_scenarios(
            tools=enriched_tools_list,
            resources=resources_list,
            num_scenarios=expected_total,
            show_preview=False,  # We'll preview separately for train/val
            generator_model=INPUT_GENERATION_MODEL,
            generator_api_key=_openrouter_key,
        )
        # Convert GeneratedScenarioCollection to list of dicts for compatibility
        scenarios = [{"task": s.task, "difficulty": s.difficulty} for s in scenario_collection.scenarios]
        ok(f"Attempt {attempt} succeeded in {time.perf_counter() - t_attempt:.2f}s.")
        break
    except Exception as e:
        warn(f"Attempt {attempt} failed: {e}")
        if attempt < max_attempts:
            time.sleep(min(1.5 * attempt, 6.0))
        else:
            err("All attempts exhausted.")
            raise

# Split into train/val
ok(f"Generated {len(scenarios)} scenarios total.")
step("Shuffling scenarios and splitting into train/val ...")
random.shuffle(scenarios)

train_n = TRAINING_CONFIG["num_training_inputs"]  # noqa: F821
raw_train_scenarios = scenarios[:train_n]
raw_val_scenarios = scenarios[train_n:]

ok(f"Train: {len(raw_train_scenarios)} | Val: {len(raw_val_scenarios)}")

info("Sample (train) preview:")
preview_scenarios(raw_train_scenarios, n=min(5, len(raw_train_scenarios)))

info("Sample (val) preview:")
preview_scenarios(raw_val_scenarios, n=min(5, len(raw_val_scenarios)))

ok("Done.")

[18:45:34] OK    OPENROUTER_API_KEY found.
INFO:     127.0.0.1:50000 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50016 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50026 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50040 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50042 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50058 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50064 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50070 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50084 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50088 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50104 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50110 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50112 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50128 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50144 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50150 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50158 - "POST /mcp HTT

In [10]:
raw_train_scenarios

[{'task': 'List the top three earners and show their names, salaries, and department names. Then provide a summary and thorough analysis/report.',
  'difficulty': 3},
 {'task': 'Identify departments that have no employees assigned. Then provide a summary and thorough analysis/report.',
  'difficulty': 3},
 {'task': 'For each department, present location, budget, and the number of employees in that department. Then provide a summary and thorough analysis/report.',
  'difficulty': 4},
 {'task': 'In the Marketing department, count hires after 2020 and report the average salary of those hires. Then generate a summary and a thorough report.',
  'difficulty': 2},
 {'task': 'Find the most senior engineer (earliest hire_date) and show their name, role, and hire date. Then produce a summary and thorough analysis/report.',
  'difficulty': 2},
 {'task': 'Compute for each department its headcount and total payroll (sum of salaries). Then generate a summary and a thorough analysis/report.',
  'diff

In [11]:
raw_val_scenarios

[{'task': 'For every department, show its budget, the total budget allocated to its projects, and the remaining budget (budget minus project budgets). Then provide a summary and a thorough analysis/report.',
  'difficulty': 4},
 {'task': 'Determine which department has the most active projects. Then provide a summary and thorough analysis/report.',
  'difficulty': 4},
 {'task': 'Determine the most common job role among all employees. Then provide a summary and thorough analysis/report.',
  'difficulty': 3},
 {'task': 'Identify which department has the highest budget and report its name and location. Then produce a summary and thorough analysis/report.',
  'difficulty': 2},
 {'task': "Produce a report of all projects including the project lead's name and the department name they belong to. Then provide a summary and thorough analysis/report.",
  'difficulty': 3},
 {'task': 'For every department, calculate budget minus total salaries of employees in that department (budget variance). The

## Register model with the server

In [12]:
# @title Run this cell to train your model!

import os
import random
from dataclasses import dataclass

import weave
from dotenv import load_dotenv
from openai import AsyncOpenAI

import art
from art.local import LocalBackend
from art.rewards import ruler_score_group
from art.utils import iterate_dataset

load_dotenv()

# Optional
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    weave.init(PROJECT_NAME)
else:
    print("WANDB_API_KEY is not set. We'll skip logging metrics to Weights & Biases.")

random.seed(42)

# Declare the model
model = art.TrainableModel(
    name=MODEL_NAME,
    project=PROJECT_NAME,
    base_model=BASE_MODEL,
)

# Clear GPU memory before registering
import torch, gc
gc.collect()
torch.cuda.empty_cache()

# To run on a T4, we need to override some config defaults.
model._internal_config = art.dev.InternalModelConfig(
    init_args=art.dev.InitArgs(
        max_seq_length=MAX_SEQ_LENGTH,
        dtype="float16",
    ),
    engine_args=art.dev.EngineArgs(
        enforce_eager=True,
        gpu_memory_utilization=0.80,
    ),
)

# Initialize the server
backend = LocalBackend(
    in_process=True,
    path="./.art",
)

# Register the model with the local Backend
await model.register(backend)

print("Model created!")
print("Base model:", BASE_MODEL)
print("Model name:", MODEL_NAME)
print("Project name:", PROJECT_NAME)

WANDB_API_KEY is not set. We'll skip logging metrics to Weights & Biases.
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3. vLLM: 0.13.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.12.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
/teamspace/studios/this_studio/.cache/huggingface/hub/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1
INFO 02-16 18:47:25 [model.py:514] Resolved architecture: Qwen2ForCausalLM
WARNING 02-16 18:47:25 [model.py:1955] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 02-16 18:47:25 [model.py:2005] Casting torch.bfloat16 to torch.float16.
INFO 02-16 18:47:25 [model.py:1661] Using max model len 32768
INFO 02-16 18:47:25 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=2048.
WARNING 02-16 18:47:25 [vllm.py:622] Enforce eager set, overriding optimization level to -O0
INFO 02-16 18:47:25 [vllm.py:722] Cudagraph is disabled under eager mode
WARNING 02-16 18:47:27 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_M

(EngineCore_DP0 pid=87084) /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/art/__init__.py:37: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.
(EngineCore_DP0 pid=87084) 
(EngineCore_DP0 pid=87084) Please restructure your imports with 'import unsloth' at the top of your file.
(EngineCore_DP0 pid=87084)   import unsloth  # noqa: F401


(EngineCore_DP0 pid=87084) 🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
(EngineCore_DP0 pid=87084) 🦥 Unsloth Zoo will now patch everything to make training faster!
(EngineCore_DP0 pid=87084) INFO 02-16 18:47:53 [worker_base.py:283] Injected <class 'art.vllm.engine.WorkerExtension'> into <class 'vllm.v1.worker.gpu_worker.Worker'> for extended collective_rpc calls ['run', 'time']
(EngineCore_DP0 pid=87084) INFO 02-16 18:47:53 [parallel_state.py:1203] world_size=1 rank=0 local_rank=0 distributed_init_method=tcp://10.128.0.201:50233 backend=nccl
(EngineCore_DP0 pid=87084) INFO 02-16 18:47:53 [parallel_state.py:1411] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, PCP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=87084) INFO 02-16 18:47:54 [gpu_model_runner.py:3562] Starting to load model Qwen/Qwen2.5-3B-Instruct...
(EngineCore_DP0 pid=87084) INFO 02-16 18:47:55 [cuda.py:351] Using FLASHINFER attention backend out of potential backends: ('FLASHINFE

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:03<00:03,  3.46s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:05<00:00,  2.77s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:05<00:00,  2.87s/it]
(EngineCore_DP0 pid=87084) 


(EngineCore_DP0 pid=87084) INFO 02-16 18:48:01 [default_loader.py:308] Loading weights took 5.83 seconds
(EngineCore_DP0 pid=87084) INFO 02-16 18:48:01 [punica_selector.py:20] Using PunicaWrapperGPU.
(EngineCore_DP0 pid=87084) INFO 02-16 18:48:02 [gpu_model_runner.py:3659] Model loading took 5.9169 GiB memory and 6.699443 seconds
(EngineCore_DP0 pid=87084) INFO 02-16 18:48:05 [gpu_worker.py:375] Available KV cache memory: 5.14 GiB
(EngineCore_DP0 pid=87084) INFO 02-16 18:48:05 [kv_cache_utils.py:1291] GPU KV cache size: 149,648 tokens
(EngineCore_DP0 pid=87084) INFO 02-16 18:48:05 [kv_cache_utils.py:1296] Maximum concurrency for 32,768 tokens per request: 4.57x
(EngineCore_DP0 pid=87084) INFO 02-16 18:48:05 [kernel_warmup.py:65] Warming up FlashInfer attention.
(EngineCore_DP0 pid=87084) INFO 02-16 18:48:08 [core.py:259] init engine (profile, create kv cache, warmup model) took 5.71 seconds
(EngineCore_DP0 pid=87084) WARNING 02-16 18:48:10 [vllm.py:629] Inductor compilation was disable

## RULER in action

In [17]:
import art
from art.rewards import ruler_score_group

base_messages = [
    {"role": "system", "content": "You count numbers using numeric symbols."},
    {"role": "user", "content": "Count to 10."},
]

good_trajectory = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {"role": "assistant", "content": "1, 2, 3, 4, 5, 6, 7, 8, 9, 10"},
    ],
    reward=0,
)

mediocre_trajectory = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {"role": "assistant",
         "content": "one, two, three, four, five, six, seven, eight, nine, ten"},
    ],
    reward=0,
)

bad_trajectory = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {"role": "assistant", "content": "a, b, c, d, e, f, g, h, i, j"},
    ],
    reward=0,
)

sample_group = art.TrajectoryGroup(
    trajectories=[good_trajectory, mediocre_trajectory, bad_trajectory]
)

judged_group = await ruler_score_group(
    sample_group, "openrouter/openai/gpt-4o-mini", debug=True
)

(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found


(APIServer pid=7894) (APIServer pid=7894) /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/aiohttp/connector.py:993: DeprecationWarning: enable_cleanup_closed ignored because https://github.com/python/cpython/pull/118960 is fixed in Python version sys.version_info(major=3, minor=12, micro=11, releaselevel='final', serial=0)
(APIServer pid=7894) (APIServer pid=7894)   super().__init__(


(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer

[RULER] Pretty-printed LLM choice JSON:

{
    'scores': [
        {
            'trajectory_id': '1',
            'explanation': 'This trajectory successfully counts to 10 using numeric symbols as requested, achieving
the goal perfectly.',
            'score': 1
        },
        {
            'trajectory_id': '2',
            'explanation': 'This trajectory counts to 10 but uses word representations instead of numeric symbols, 
which does not fulfill the goal as specified.',
            'score': 0.2
        },
        {
            'trajectory_id': '3',
            'explanation': 'This trajectory does not count at all but instead uses letters. It does not achieve the
goal.',
            'score': 0
        }
    ]
}

(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50730 - "GET /v1/models HTTP/1.1" 404 Not Found
(APIServer pid=7894) (APIServer pid=7894) INFO:     127.0.0.1:50958 - "GET /v1/models HTTP/1.1" 404 Not Found


## Define rollout function

In [14]:
@dataclass
class McpScenario:
    """A scenario for MCP agent evaluation against a remote Smithery server."""

    task_description: str
    max_turns: int = MAX_TURNS


@weave.op()
async def rollout(
    model: art.Model,
    scenario: McpScenario,
    debug: bool = False,
) -> art.Trajectory:
    """Run an MCP agent rollout against the remote Smithery MCP server."""
    traj = art.Trajectory(
        messages_and_choices=[],
        reward=0,
        metadata={"task": scenario.task_description},
        metrics={
            "task_completed": False,
            "success": False,
            "ran_out_of_turns": False,
        },
        scenario=scenario,
    )

    # Discover available tools from the remote server
    tools_result, _resources_result = await list_tools_and_resources()
    tool_names = [t.name for t in tools_result.tools]
    log("rollout: discovered tools", count=len(tool_names), names=tool_names)

    # Convert to OpenAI tool format
    tool_schemas = []
    for tool in tools_result.tools:
        tool_schema = {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description or f"MCP tool: {tool.name}",
                "parameters": tool.inputSchema or {"type": "object", "properties": {}},
            },
        }
        tool_schemas.append(tool_schema)

    # Add completion tool schema
    tool_schemas.append(
        {
            "type": "function",
            "function": {
                "name": "complete_task",
                "description": "Complete the task with a summary",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "summary": {
                            "type": "string",
                            "description": "Summary of accomplishments",
                        }
                    },
                    "required": ["summary"],
                },
            },
        }
    )

    traj.tools = tool_schemas

    # Initialize conversation
    system_prompt = (
        f"You are a database agent with access to a live SQL database. "
        f"The database is working correctly. DO NOT claim there are errors "
        f"unless a tool explicitly returns an error message. "
        f"ALWAYS use tools to answer questions — never simulate or guess. "
        f"Start by calling list_tables, then describe_table, then run_query. "
        f"Call 'complete_task' with your answer when done. "
        f"You have {scenario.max_turns} turns."
    )

    traj.messages_and_choices = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": f"Please complete this task: {scenario.task_description}",
        },
    ]

    num_turns = 0
    task_completed = False

    # Main interaction loop
    while num_turns < scenario.max_turns and not task_completed:
        num_turns += 1

        try:
            # === Log request ===
            last_user = next(
                (m for m in reversed(traj.messages()) if m["role"] == "user"), None
            )
            log(
                "LLM request",
                step=num_turns,
                model=(model.inference_model_name or model.name),
                tools=len(tool_schemas),
                last_user=(last_user["content"][:160] + "..." if last_user else None),
            )

            # Get LLM response
            async with traj.track_duration("llm_completion"):
                openai_client = AsyncOpenAI(
                    api_key=model.inference_api_key,
                    base_url=model.inference_base_url,
                )

                # We also log the request body (without huge params)
                req_preview = {
                    "model": model.inference_model_name
                    if model.inference_model_name
                    else model.name,
                    "messages_len": len(traj.messages()),
                    "tools_len": len(tool_schemas),
                }
                log_json("LLM request (preview)", req_preview)

                response = await openai_client.chat.completions.create(
                    model=model.inference_model_name
                    if model.inference_model_name
                    else model.name,
                    messages=traj.messages(),
                    tools=tool_schemas,
                    max_completion_tokens=8000,
                )

            # === Log response ===
            choice = response.choices[0]

            finish_reason = getattr(choice, "finish_reason", None)
            msg = choice.message
            has_tools = bool(getattr(msg, "tool_calls", None))
            content_preview = (
                (msg.content[:200] + "...")
                if isinstance(msg.content, str) and msg.content
                else str(msg.content)[:200]
            )
            log(
                "LLM response parsed",
                finish_reason=finish_reason,
                has_tool_calls=has_tools,
                content_preview=content_preview,
            )

            traj.messages_and_choices.append(choice)

            # Handle tool calls
            if msg.tool_calls:
                for tool_call in msg.tool_calls:
                    try:
                        log(
                            "Tool call received",
                            name=tool_call.function.name,
                            raw_args=tool_call.function.arguments,
                        )
                        tool_args = json.loads(tool_call.function.arguments or "{}")

                        if tool_call.function.name == "complete_task":
                            traj.metrics["task_completed"] = True
                            task_completed = True
                            traj.logs.append(
                                f"Task completion attempted with summary: {tool_args.get('summary', '')}"
                            )
                            # We still append a tool message for completeness
                            traj.messages_and_choices.append(
                                {
                                    "role": "tool",
                                    "tool_call_id": tool_call.id,
                                    "content": "Task marked complete.",
                                }
                            )
                        else:
                            # 🔧 Call MCP tool through remote Smithery session
                            result = await call_mcp_tool(
                                tool_call.function.name, tool_args
                            )

                            content_text = get_content_text(result)
                            log(
                                "Tool result",
                                name=tool_call.function.name,
                                len=len(content_text),
                            )

                            if len(content_text) > 20000:
                                # print(
                                #     f"Tool call result for {tool_call.function.name} is too long: {len(content_text)}"
                                # )
                                # print(f"Args: {tool_args}")
                                # print(content_text[:1000])
                                # print(content_text[-1000:])
                                raise Exception(
                                    f"Tool call result for {tool_call.function.name} is too long: {len(content_text)}"
                                )

                            # Add tool response
                            traj.messages_and_choices.append(
                                {
                                    "role": "tool",
                                    "tool_call_id": tool_call.id,
                                    "content": content_text,
                                }
                            )

                    except Exception as e:
                        traceback.print_exc()
                        traj.logs.append(f"Tool call error: {e}")

                        # Add error response
                        traj.messages_and_choices.append(
                            {
                                "role": "tool",
                                "tool_call_id": tool_call.id,
                                "content": f"Error: {str(e)}",
                            }
                        )
            else:
                # No tool calls — log and continue (RULER will likely give 0)
                log(
                    "LLM returned no tool_calls; skipping tool execution",
                    turn=num_turns,
                )
                # You can consider breaking here or letting it try another turn
                # break

        except Exception as e:
            traceback.print_exc()
            traj.logs.append(f"Error in turn {num_turns}: {e}")
            break

    if not task_completed and num_turns == scenario.max_turns:
        traj.metrics["ran_out_of_turns"] = True

    traj.metrics["num_turns"] = num_turns

    return traj.finish()

## Training pipeline

In [15]:
# =============== Training code ===============

print(
    f"Using config: max_turns={MAX_TURNS}, rollouts_per_group={TRAINING_CONFIG['rollouts_per_group']}, "
    f"groups_per_step={TRAINING_CONFIG['groups_per_step']}, num_epochs={TRAINING_CONFIG['num_epochs']}, "
    f"learning_rate={TRAINING_CONFIG['learning_rate']}"
)

await model.register(backend)

train_scenarios = [
    McpScenario(
        task_description=scenario["task"],
        max_turns=MAX_TURNS,
    )
    for scenario in raw_train_scenarios
]

# Create dataset iterator using raw scenarios
train_iterator = iterate_dataset(
    train_scenarios,
    groups_per_step=TRAINING_CONFIG["groups_per_step"],
    num_epochs=TRAINING_CONFIG["num_epochs"],
    initial_step=await model.get_step(),  # Resume from checkpoint
)

# Main training loop using iterate_dataset
for batch in train_iterator:
    print("Gathering trajectory groups with RULER scoring...")

    # Use gather_trajectory_groups with ruler_score_group
    groups = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(
                rollout(model, scenario, False)
                for _ in range(TRAINING_CONFIG["rollouts_per_group"])
            )
            for scenario in batch.items
        ),
        pbar_desc=f"train gather step {batch.step}",
    )

    scored_groups = []
    for group in groups:
        # Use RULER to assign relative scores to each trajectory
        judged_group = await ruler_score_group(
            group, judge_model=RULER_MODEL, debug=True, swallow_exceptions=True
        )
        scored_groups.append(judged_group)

    print("starting train")
    await model.train(
        scored_groups,
        config=art.TrainConfig(learning_rate=TRAINING_CONFIG["learning_rate"]),
    )

(APIServer pid=75472) Using config: max_turns=3, rollouts_per_group=4, groups_per_step=2, num_epochs=1, learning_rate=1e-05


Iterating dataset:   0%|          | 0/8 [00:00<?, ?batch/s]

(APIServer pid=75472) (APIServer pid=75472) Gathering trajectory groups with RULER scoring...


train gather step 0:   0%|          | 0/8 [00:00<?, ?it/s]

[weave.trace.op|WARNING]Warning: Traces will not be logged. Call weave.init to log your traces to a project.
 (subsequent messages of this type will be suppressed)


...

{
    'scores': [
        {
            'trajectory_id': '1',
            'explanation': 'The trajectory attempted to complete the task but encountered an SQL error while trying
to count hires per year. It correctly identified the relevant table and column, but failed to execute the required 
query due to a function limitation. Progress made, but goal not achieved.',
            'score': 0.4
        },
        {
            'trajectory_id': '2',
            'explanation': "This trajectory mirrored the first one, correctly identifying the 'employees' table and
the 'hire_date' column. However, it also did not succeed in executing the SQL query due to the same limitation. 
Overall, progress was made but the goal was not fully achieved.",
            'score': 0.4
        },
        {
            'trajectory_id': '3',
            'explanation': 'Similar to trajectories 1 and 2, this trajectory successfully identified the correct 
table and column but faced an SQL function limitation while executing the query. Progress towards the goal is 
noted, but it was ultimately not achieved.',
            'score': 0.4
        },
        {
            'trajectory_id': '4',
            'explanation': "This trajectory made multiple attempts to execute an invalid query first, then shifted 
focus to the 'employees' table as expected. Despite facing SQL function limitations as well, it demonstrated a 
better recognition of process by refining its approach. Task marked complete without achieving the analytical 
output, but better problem-solving showed higher engagement. Partial progress made.",
            'score': 0.5
        }
    ]
}

[RULER] Pretty-printed LLM choice JSON:

{
    'scores': [
        {
            'trajectory_id': '1',
            'explanation': "The trajectory makes good progress by accurately listing available tables and 
describing the structure of the 'projects' table. However, it does not execute the count query for active projects,
which means it does not achieve the goal. Therefore, the score reflects its progress without completion.",
            'score': 0.5
        },
        {
            'trajectory_id': '2',
            'explanation': "This trajectory follows a similar structure as trajectory 1, listing and describing the
necessary tables appropriately. It progresses by running a query to extract department_ids from active projects, 
which shows partial success in achieving the goal, although it doesn't yet count the active projects by department.
The score is slightly higher as it defeats the previous trajectory by making progress towards utilizing the data.",
            'score': 0.6
        },
        {
            'trajectory_id': '3',
            'explanation': 'Trajectory 3 successfully describes the necessary tables and their columns in detail. 
Unlike the previous trajectories, it constructs and runs a final SQL query to count active projects grouped by 
department, achieving the task as outlined in the user prompt. Therefore, this trajectory receives a high score for
completing the task correctly.',
            'score': 1.0
        },
        {
            'trajectory_id': '4',
            'explanation': "This trajectory correctly identifies the tables and describes the relevant 'projects' 
table with its columns. It attempts to form a query to filter and count active projects by department but does not 
execute a proper SELECT query, making it fall short of completing the task. The lack of a successful query 
execution results in a moderate score.",
            'score': 0.4
        }
    ]
}

(APIServer pid=75472) (APIServer pid=75472) starting train
(APIServer pid=75472) (APIServer pid=75472) Packed 8 trajectories into 4 sequences of length 2048


train:   0%|          | 0/4 [00:00<?, ?it/s]

## Test your model

In [16]:
# @title Test Your Model!

# Generate test inputs
print("Generating test inputs...")
val_scenarios = [
    McpScenario(
        task_description=scenario["task"],
        max_turns=MAX_TURNS,
    )
    for scenario in raw_val_scenarios
]

print(f"\n🧪 Testing the trained model on {len(val_scenarios)} new inputs:\n")
print("=" * 80)

for i, scenario in enumerate(val_scenarios):
    print(f"\nTest {i + 1}:")
    print(f"Input: {scenario.task_description}")

    # Run the model
    result_trajectory = await rollout(model, scenario)

    # Extract the model's response
    messages = result_trajectory.messages()
    model_response = messages[-1]["content"] if messages else "No response"

    print(f"Model output: {model_response}")
    print("-" * 80)

print("\n🎉 Testing completed!")
print(
    f"\nYour model '{MODEL_NAME}' has been trained to use the MCP server at:"
)
print(MCP_SERVER_URL)
print("\nTo use this model in production:")
print("1. The model checkpoint is saved in ./.art/")
print("2. You can load it using the vLLM library")
print(
    "3. Or continue training with more examples by adjusting the configuration at the top"
)

(APIServer pid=75472) (APIServer pid=75472) Generating test inputs...
(APIServer pid=75472) (APIServer pid=75472) 
(APIServer pid=75472) (APIServer pid=75472) 🧪 Testing the trained model on 8 new inputs:
(APIServer pid=75472) (APIServer pid=75472) 
(APIServer pid=75472) (APIServer pid=75472) ================================================================================
(APIServer pid=75472) (APIServer pid=75472) 
(APIServer pid=75472) (APIServer pid=75472) Test 1:
(APIServer pid=75472) (APIServer pid=75472) Input: For every department, show its budget, the total budget allocated to its projects, and the remaining budget (budget minus project budgets). Then provide a summary and a thorough analysis/report.
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50280 - "POST /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50294 - "POST /mcp HTTP/1.1" 202 Accepted
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50308 - "GET 

(APIServer pid=75472) (APIServer pid=75472) [19:02:27] LLM response parsed | finish_reason='tool_calls' has_tool_calls=True content_preview="To accomplish this task, we first need to identify the tables that might contain the budget information and project allocations. Let's start by listing all the tables in the database.\n..."
(APIServer pid=75472) (APIServer pid=75472) [19:02:27] Tool call received | name='list_tables' raw_args='{}'
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50352 - "POST /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50354 - "POST /mcp HTTP/1.1" 202 Accepted
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50360 - "GET /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50376 - "POST /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50380 - "POST /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     1

In [19]:
raw_val_scenarios

[{'task': 'For every department, show its budget, the total budget allocated to its projects, and the remaining budget (budget minus project budgets). Then provide a summary and a thorough analysis/report.',
  'difficulty': 4},
 {'task': 'Determine which department has the most active projects. Then provide a summary and thorough analysis/report.',
  'difficulty': 4},
 {'task': 'Determine the most common job role among all employees. Then provide a summary and thorough analysis/report.',
  'difficulty': 3},
 {'task': 'Identify which department has the highest budget and report its name and location. Then produce a summary and thorough analysis/report.',
  'difficulty': 2},
 {'task': "Produce a report of all projects including the project lead's name and the department name they belong to. Then provide a summary and thorough analysis/report.",
  'difficulty': 3},
 {'task': 'For every department, calculate budget minus total salaries of employees in that department (budget variance). The

In [20]:
scenario = McpScenario(
    task_description="Which department has the most active projects?",
    max_turns=MAX_TURNS,
)

# Run the model
result_trajectory = await rollout(model, scenario)

# Extract the model's response
messages = result_trajectory.messages()
model_response = messages[-1]["content"] if messages else "No response"

print(f"Model output: {model_response}")
print("-" * 80)

(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50168 - "POST /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50176 - "POST /mcp HTTP/1.1" 202 Accepted
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50184 - "GET /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50186 - "POST /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50198 - "POST /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) INFO:     127.0.0.1:50206 - "DELETE /mcp HTTP/1.1" 200 OK
(APIServer pid=75472) (APIServer pid=75472) [19:19:13] rollout: discovered tools | count=3 names=['list_tables', 'describe_table', 'run_query']
(APIServer pid=75472) (APIServer pid=75472) [19:19:13] LLM request | step=1 model='sql-agent-3b@0' tools=4 last_user='Please complete this task: Which department has the most active projects?...'
(APIServer pid=75472) (APIServer pid=75472) [19:19:13

In [ ]:
# @title Upload to Hugging Face 🤗

# Adapted from Unsloth Notebooks (https://github.com/unslothai/notebooks), licensed under GNU LGPL v3.0.
# © Unsloth contributors. Modifications © 2025 OpenPipe, Inc.
# See THIRD-PARTY-NOTICES and licenses/LGPL-3.0.txt for details.

import torch
from unsloth import FastLanguageModel

lora_model_path = (
    f".art/{model.project}/models/{model.name}/checkpoints/{await model.get_step():04d}"
)

peft_model, peft_tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_path,
    max_seq_length=16384,
    dtype=torch.bfloat16,
    load_in_4bit=True,
)

UPLOAD_MODEL = False  # Set True when you're ready to upload your model to Hugging Face
HF_ACCOUNT = "your_hf_account"
HF_TOKEN = "your_hf_token"

if UPLOAD_MODEL:
    peft_model.push_to_hub_merged(
        f"{HF_ACCOUNT}/{model.name}", peft_tokenizer, token=HF_TOKEN
    )

### Next Steps

Congratulations! You've successfully trained a custom model for your task using only:
- A pre-built MCP server
- Example inputs (no outputs needed!)
- RULER's automatic evaluation

Here are some ways to improve results:

1. **More diverse inputs**: Generate more varied input examples
2. **Longer training**: Increase the number of training steps
3. **More comparisons**: Increase `rollouts_per_group` for better RULER comparisons
4. **MCP server refinement**: Add better tools and resources to the server
5. **Hyperparameter tuning**: Adjust learning rate, batch size, etc.

Remember: RULER learns what "good" means from your MCP server alone - no labeled data required!

For more advanced use cases, check out the [ART documentation](https://art.openpipe.ai).